# Unified Model with Multiple Strategic Agents: Finite-Horizon Analysis

James Yu, 23 October 2022

In [1]:
from collections import namedtuple #,defaultdict
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv as npinv
from numpy.linalg import matrix_power as nppow

In [2]:
def finite_solution(A, delta, c, T, x_0, M = 1):
    n = len(x_0)
    eps = np.finfo(np.float64).eps
    I = np.identity(n)
    eigvals, U = np.linalg.eig(A)
    chi_0 = U.T @ x_0
    D = np.diag(eigvals)
    K_t = I
    K_sequence = [K_t]
    
    for i in range(T): # generate solution matrices
        K_t_new = I + c * delta * (delta * K_t + c * I) @ nppow(npinv(M * delta * K_t + c * I), 2) @ K_t @ nppow(D, 2)
        K_sequence.insert(0, K_t_new)
        #if np.allclose(K_t, K_t_new, rtol = eps, atol = eps) and i >= T: break
        K_t = K_t_new

    chi_ts = [chi_0]
    chi_var = chi_0
    gamma_ts = []
    L_ts = []
    #K_ss = K_sequence[0]
    #L_ss = -inv(B.T @ K_ss @ B + c * I / delta) @ B.T @ K_ss @ A
    
    payoff = 0
    stage_payoffs = []
    discounted_stage_payoffs = []
    cumulative_payoffs = []

    for i, K in enumerate(K_sequence[1:]): # while True
        L_t = -delta * npinv(c * I + M * delta * K) @ K @ D
        L_ts.append(L_t)
        gamma_t = L_t @ chi_var
        gamma_ts.append(gamma_t)
        p = -(chi_var.T @ chi_var + c * gamma_t.T @ gamma_t).item()
        payoff += delta**i * p
        stage_payoffs.append(p)
        discounted_stage_payoffs.append(delta**i * p)
        cumulative_payoffs.append(payoff)
        chi_var_new = D @ chi_var + M * gamma_t
        chi_ts.append(chi_var_new)
        #if np.allclose(chi_var, chi_var_new, rtol = eps, atol = eps): break
        chi_var = chi_var_new
        # i += 1
        
    Results = namedtuple("Results", ["chi_ts", "gamma_ts", "K_ts", "L_ts", "payoff", "stage_payoffs", "discounted_stage_payoffs", "cumulative_payoffs"])
    return Results(chi_ts, gamma_ts, K_sequence, L_ts, payoff, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs)#, K_ss, L_ss

### Baseline: $M = 1$ strategic agents:

In [3]:
A = np.array([
    [0.9, 0.1],
    [0.1, 0.9]
])
result = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 1)
# K_t results
print("Index 1 Index 2")
for K in result.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.3545766763101257 1.2198673635043773
1.3545766758474271 1.2198673634799293
1.3545766697698922 1.2198673630464518
1.3545765899416025 1.219867355360585
1.3545755413994027 1.2198672190847089
1.3545617689358052 1.2198648028201033
1.3543808878763623 1.2198219618861206
1.3520084566596196 1.2190627253064168
1.3214285714285714 1.2057142857142857
1.0 1.0


In [4]:
# L_t results
print("Index 1 Index 2")
for L in result.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.7091533526202515 -0.549668408760943
-0.7091533516948545 -0.5496684086998234
-0.7091533395397842 -0.5496684076161293
-0.7091531798832047 -0.5496683884014626
-0.7091510827988052 -0.5496680477117718
-0.7091235378716104 -0.5496620070502581
-0.7087617757527246 -0.5495549047153017
-0.7040169133192389 -0.5476568132660419
-0.6428571428571429 -0.5142857142857143


### $M = 2$

In [5]:
result_2 = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 2)
# K_t results
print("Index 1 Index 2")
for K in result_2.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.1200638417170201 1.0766350168801482
1.1200638417170201 1.0766350168801482
1.1200638417170201 1.0766350168801482
1.120063841717003 1.0766350168801468
1.1200638417146014 1.0766350168798946
1.1200638413745252 1.0766350168291208
1.1200637932224402 1.076635006610833
1.1200569753202527 1.0766329501743985
1.1190926275992439 1.076219281663516
1.0 1.0


In [6]:
# L_t results
print("Index 1 Index 2")
for L in result_2.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.40064046816757 -0.31796362855596505
-0.40064046816757 -0.31796362855596505
-0.40064046816756876 -0.3179636285559649
-0.40064046816739807 -0.3179636285559497
-0.4006404681432253 -0.3179636285528743
-0.40064046472054216 -0.3179636279339557
-0.4006399800970261 -0.31796350337603474
-0.40057138561010447 -0.31793843986857745
-0.39130434782608703 -0.31304347826086965


### $M = 3$

In [7]:
result_3 = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 3)
# K_t results
print("Index 1 Index 2")
for K in result_3.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.06135663917094 1.0393063358105732
1.06135663917094 1.0393063358105732
1.06135663917094 1.0393063358105732
1.06135663917094 1.0393063358105732
1.0613566391709486 1.0393063358105736
1.0613566391677058 1.0393063358102146
1.0613566403747612 1.0393063360172352
1.0613561910770124 1.0393062166883937
1.0615234375 1.039375
1.0 1.0


In [8]:
# L_t results
print("Index 1 Index 2")
for L in result_3.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.2838136189879178 -0.22633750537592567
-0.2838136189879178 -0.22633750537592567
-0.2838136189879178 -0.22633750537592567
-0.2838136189879182 -0.22633750537592567
-0.2838136189877894 -0.22633750537591385
-0.2838136190357405 -0.22633750538273212
-0.2838136011870781 -0.22633750145258447
-0.28382024427746677 -0.22633976673408818
-0.28125 -0.225


### $M = 4$

In [9]:
result_4 = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 4)
# K_t results
print("Index 1 Index 2")
for K in result_4.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.0373229161282222 1.0239217067980593
1.0373229161282222 1.0239217067980593
1.0373229161282222 1.0239217067980593
1.037322916128222 1.0239217067980593
1.037322916128264 1.0239217067980624
1.037322916117928 1.0239217067968862
1.0373229186689057 1.0239217072430158
1.0373222890728773 1.0239215380284017
1.0374776918500892 1.0239857227840572
1.0 1.0


In [10]:
# L_t results
print("Index 1 Index 2")
for L in result_4.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.22047961763849405 -0.17611151989772883
-0.22047961763849405 -0.17611151989772883
-0.22047961763849402 -0.17611151989772883
-0.22047961763849505 -0.17611151989772889
-0.22047961763823565 -0.1761115198977047
-0.22047961770225974 -0.17611151990686988
-0.2204796019007474 -0.1761115164305681
-0.220483501664583 -0.17611283495318908
-0.21951219512195125 -0.17560975609756102


### $M = 10$

In [11]:
result_10 = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 10)
# K_t results
print("Index 1 Index 2")
for K in result_10.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.0069684320011412 1.004462588025787
1.0069684320011412 1.004462588025787
1.0069684320011412 1.004462588025787
1.0069684320011412 1.004462588025787
1.0069684320011414 1.004462588025787
1.0069684320010774 1.00446258802578
1.0069684320378745 1.004462588032022
1.0069684108514427 1.0044625824412194
1.0069806094182825 1.004467590027701
1.0 1.0


In [12]:
# L_t results
print("Index 1 Index 2")
for L in result_10.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.09477135987458439 -0.0758071996490281
-0.09477135987458439 -0.0758071996490281
-0.09477135987458439 -0.0758071996490281
-0.09477135987458438 -0.0758071996490281
-0.09477135987458407 -0.07580719964902807
-0.09477135987476515 -0.07580719964905276
-0.09477135977050752 -0.07580719962693888
-0.09477141979850415 -0.07580721943387475
-0.09473684210526316 -0.07578947368421053


### $M = 100$

In [13]:
result_100 = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 100)
# K_t results
print("Index 1 Index 2")
for K in result_100.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.0000769186813623 1.000049228427797
1.0000769186813623 1.000049228427797
1.0000769186813623 1.000049228427797
1.0000769186813623 1.000049228427797
1.0000769186813623 1.000049228427797
1.0000769186813623 1.000049228427797
1.0000769186813623 1.000049228427797
1.0000769186813079 1.0000492284277829
1.0000769207289155 1.000049229266506
1.0 1.0


In [14]:
# L_t results
print("Index 1 Index 2")
for L in result_100.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.009944755607064388 -0.007955803268687334
-0.009944755607064388 -0.007955803268687334
-0.009944755607064388 -0.007955803268687334
-0.009944755607064388 -0.007955803268687334
-0.009944755607064388 -0.007955803268687334
-0.009944755607064388 -0.007955803268687334
-0.009944755607064384 -0.007955803268687334
-0.009944755607176873 -0.007955803268724197
-0.009944751381215469 -0.007955801104972375


### $M = 1000000$ (limit case)

In [15]:
result_1000000 = finite_solution(A, 0.9, 0.5, 10-1, np.array([[1.0, 1.0]], ndmin = 2).T, M = 1000000)
# K_t results
print("Index 1 Index 2")
for K in result_1000000.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0000000000007778 1.0000000000004978
1.0 1.0


In [16]:
# L_t results
print("Index 1 Index 2")
for L in result_1000000.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
-9.999994444447531e-07 -7.999995555558025e-07
